# Comparing KPNO and NERSC GFA file reductions

## Imports

In [ ]:
%matplotlib inline
import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

## Adjust these values as needed

In [ ]:
night = 20240113  # Identify the specific file to compare.
nightMin = 20230830  # Start date for comparison.
nightMax = 20400101  # Arbitrary date in the future.
plot_value = 'TRANSPARENCY'  # Compare this value.

## Load files

In [ ]:
with fits.open(os.path.join(os.environ['DESI_ROOT'], 'survey', 'GFA', f'offline_matched_coadd_ccds_SV3-thru_{night:d}.fits'), mode='readonly') as hdulist:
    kpno = hdulist[2].data
with fits.open(os.path.join(os.environ['DESI_ROOT'], 'users', 'desi', 'GFA', 'summary_files', f'offline_matched_coadd_ccds_SV3-thru_{night:d}.fits'), mode='readonly') as hdulist:
    nersc = hdulist[2].data

## Set range of nights

In [ ]:
i = np.where((nersc['NIGHT'] >= nightMin) & (nersc['NIGHT'] < nightMax))[0]
assert (nersc[i]['EXPID'] == kpno[i]['EXPID']).all()  # Are the EXPIDs identical?
assert (nersc[i]['EXPID'] == sorted(nersc[i]['EXPID'])).all()  # Are the EXPIDs in strictly ascending order?
nightMax = nersc[i]['NIGHT'].max()

## Compute Difference

In [ ]:
relative_difference = (kpno[i][plot_value] - nersc[i][plot_value])/nersc[i][plot_value]
yMin, yMax = relative_difference.min(), relative_difference.max()
yMin, yMax

## Plot relative difference versus EXPID

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,9), dpi=100)
pl = axes.plot(nersc[i]['EXPID'], relative_difference, 'k-', label=plot_value)
foo = axes.set_ylim([-0.06, 0.05])
foo = axes.set_xlabel("EXPID")
foo = axes.set_ylabel("Relative Difference [(KPNO - NERSC)/NERSC]")
foo = axes.set_title(f"Comparing {plot_value}, {nightMin:d} to {nightMax:d}")
foo = axes.grid(True)
figure.savefig(f'kpno_nersc_gfa_relative_{plot_value.lower()}.png')

## Relative difference histogram

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,9), dpi=100)
n, x = np.histogram(relative_difference, bins=50)
pl = axes.step(x[0:50], n, 'k-') # , width=1, align='edge', color='w', edgecolor='k', label=plot_value)
foo = axes.set_xlim([-0.06, 0.05])
foo = axes.set_xlabel("Relative Difference [(KPNO - NERSC)/NERSC]")
foo = axes.set_ylabel("N")
foo = axes.set_title(f"Comparing {plot_value}")
foo = axes.grid(True)
figure.savefig(f'kpno_nersc_gfa_histogram_{plot_value.lower()}.png')